In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.special as spec
import scipy.stats as st
import statsmodels.formula.api as smf

from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR

%matplotlib inline

In [2]:
data_dir = "https://raw.githubusercontent.com/chirayukong/infsci2725-spring-2018/master/assignment-03/Retention.txt"
df = pd.read_table(data_dir, sep="\t")
df.head()

,spend,apret,top10,rejr,tstsc,pacc,strat,salar
0,9855,52.50,15,29.474,65.063,36.887,12.0,60800
1,10527,64.25,36,22.309,71.063,30.970,12.8,63900
2,7904,37.75,26,25.853,60.750,41.985,20.3,57800
3,6601,57.00,23,11.296,67.188,40.289,17.0,51200
4,7251,62.00,17,22.635,56.250,46.780,18.1,48000


In [3]:
df.describe(include='all')

,spend,apret,top10,rejr,tstsc,pacc,strat,salar
count,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000,170.000000
mean,10974.511765,56.721076,38.458824,30.654218,66.164165,43.173106,16.086471,61357.647059
std,5500.065580,18.077097,23.406393,17.098104,6.975306,13.105195,4.006503,9802.786457
min,4125.000000,18.750000,8.000000,0.000000,48.125000,8.964000,7.200000,38640.000000
25%,7371.750000,45.374750,22.000000,19.171000,61.111000,33.903750,13.400000,54650.000000
50%,9265.000000,55.708500,30.000000,27.390500,64.781500,40.850500,16.000000,61150.000000
75%,12838.000000,68.687500,49.500000,36.807500,70.453250,51.773250,18.575000,67100.000000
max,35863.000000,95.250000,98.000000,84.067000,87.500000,76.253000,29.200000,87900.000000


In [4]:
df.median()

spend     9265.0000
apret       55.7085
top10       30.0000
rejr        27.3905
tstsc       64.7815
pacc        40.8505
strat       16.0000
salar    61150.0000
dtype: float64

In [5]:
lm = smf.ols(formula='apret ~ tstsc + pacc	+ strat + tstsc*pacc', data=df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  apret   R-squared:                       0.659
Model:                            OLS   Adj. R-squared:                  0.650
Method:                 Least Squares   F-statistic:                     79.58
Date:                Thu, 05 Apr 2018   Prob (F-statistic):           1.72e-37
Time:                        19:10:53   Log-Likelihood:                -641.46
No. Observations:                 170   AIC:                             1293.
Df Residuals:                     165   BIC:                             1309.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -4.4215     26.205     -0.169      0.8

# Stacking

In [6]:
X = df[['tstsc', 'pacc', 	'strat']]
X['tstsc_pacc'] = df['tstsc'] * df['pacc']
lm = smf.ols(formula='tstsc_pacc ~ tstsc*pacc', data=X).fit()
print(lm.summary())
X.head()

                            OLS Regression Results                            
Dep. Variable:             tstsc_pacc   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.013e+31
Date:                Thu, 05 Apr 2018   Prob (F-statistic):               0.00
Time:                        19:10:53   Log-Likelihood:                 4336.5
No. Observations:                 170   AIC:                            -8665.
Df Residuals:                     166   BIC:                            -8652.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept   3.865e-12   4.93e-12      0.784      0.4

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,tstsc,pacc,strat,tstsc_pacc
0,65.063,36.887,12.0,2399.978881
1,71.063,30.970,12.8,2200.821110
2,60.750,41.985,20.3,2550.588750
3,67.188,40.289,17.0,2706.937332
4,56.250,46.780,18.1,2631.375000


In [7]:
y = df['apret']
y.head()

0    52.50
1    64.25
2    37.75
3    57.00
4    62.00
Name: apret, dtype: float64

In [8]:
lr = LinearRegression()
lr.fit(X, y)
y_LR = lr.predict(X)
diff = y - y_LR
diff.describe()

count    1.700000e+02
mean    -6.185901e-15
std      1.056217e+01
min     -2.528376e+01
25%     -6.321386e+00
50%      2.216940e-02
75%      6.503852e+00
max      2.598902e+01
Name: apret, dtype: float64

In [9]:
rbf = SVR(kernel='rbf')
rbf.fit(X, y)
y_RBF = rbf.predict(X)
diff = y - y_RBF
diff.describe()

count    170.000000
mean       0.785812
std       17.279108
min      -36.188307
25%       -9.563557
50%       -0.099675
75%       11.749192
max       38.311693
Name: apret, dtype: float64

In [10]:
stacking_df = df.copy()
stacking_df['LR'] = y_LR
stacking_df['RBF'] = y_RBF
stacking_df.head()

,spend,apret,top10,rejr,tstsc,pacc,strat,salar,LR,RBF
0,9855,52.50,15,29.474,65.063,36.887,12.0,60800,58.266107,54.938307
1,10527,64.25,36,22.309,71.063,30.970,12.8,63900,69.165170,56.938307
2,7904,37.75,26,25.853,60.750,41.985,20.3,57800,45.631300,54.938306
3,6601,57.00,23,11.296,67.188,40.289,17.0,51200,59.006532,56.900294
4,7251,62.00,17,22.635,56.250,46.780,18.1,48000,36.730226,56.938307


In [11]:
lm = smf.ols(formula='apret ~ tstsc + pacc	+ strat + tstsc*pacc + LR + RBF', data=stacking_df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  apret   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     138.3
Date:                Thu, 05 Apr 2018   Prob (F-statistic):           6.18e-57
Time:                        19:10:55   Log-Likelihood:                -592.39
No. Observations:                 170   AIC:                             1197.
Df Residuals:                     164   BIC:                             1216.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -86.0590      8.306    -10.361      0.0

In [12]:
lm = smf.ols(formula='apret ~ LR*tstsc + LR*pacc	+ LR*strat + LR*tstsc*pacc + RBF*tstsc + RBF*pacc	+ RBF*strat + RBF*tstsc*pacc', data=stacking_df).fit()
print(lm.summary())

                            OLS Regression Results                            
Dep. Variable:                  apret   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.815
Method:                 Least Squares   F-statistic:                     58.11
Date:                Thu, 05 Apr 2018   Prob (F-statistic):           6.40e-53
Time:                        19:10:56   Log-Likelihood:                -582.77
No. Observations:                 170   AIC:                             1194.
Df Residuals:                     156   BIC:                             1237.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       -701.4760    458.180     -1.